In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import xgboost as xgb

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, label_binarize
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [15]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [16]:
def create_convnext_like_feature_extractor(input_dim, sliding_window_size=3):
    input_layer = Input(shape=(input_dim,))
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)
    x = reshaped_input

    # Simulated ConvNeXt block
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 8)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)

    # Flatten the output for feature extraction
    feature_output = Flatten()(x)

    # Reconstruction head for self-supervised learning
    reconstruction_output = Dense(input_dim, activation='linear')(feature_output)

    # Create the model
    model = Model(inputs=input_layer, outputs=reconstruction_output)
    return model

In [ ]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    train_X = train_data.drop(['label'], axis=1)
    if (is_string_labels):
        train_y = train_data['label'].map(label_mapping)
    else:
        train_y = train_data['label'] - 1
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    test_X = test_data.drop(['label'], axis=1)
    if (is_string_labels):
        test_y = test_data['label'].map(label_mapping)
    else:
        test_y = test_data['label'] - 1
    test_y_binarized = label_binarize(test_y, classes=range(len(np.unique(test_y))))
    del test_data
    gc.collect()

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Define XGBoost parameters
    params = {
        "objective": "multi:softprob",
        "num_class": len(train_y.unique()),
        "eval_metric": "mlogloss",
        "max_depth": 6,
        "eta": 0.3,
        "seed": 42
    }

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs} started...")
                
        # Train the NN feature extractor
        feature_extractor = create_convnext_like_feature_extractor(train_X.shape[1])
        feature_extractor.compile(optimizer='adam', loss='mean_squared_error')
        feature_extractor.fit(train_X, train_X, epochs=3, batch_size=32, verbose=0)  # Self-supervised training

        # Extract features
        feature_extractor_truncated = Model(inputs=feature_extractor.input, outputs=feature_extractor.layers[-2].output)
        train_features = feature_extractor_truncated.predict(train_X, verbose=0)
        test_features = feature_extractor_truncated.predict(test_X, verbose=0)
        del feature_extractor
        gc.collect()

        train_combined = np.hstack((train_X, train_features))
        test_combined = np.hstack((test_X, test_features))
        del train_features
        del test_features
        gc.collect()

        # Convert data to DMatrix
        dtrain = xgb.DMatrix(data=train_combined, label=train_y)
        dtest = xgb.DMatrix(data=test_combined, label=test_y)
        del train_combined
        del test_combined
        gc.collect()

        # Train XGBoost
        num_round = 100
        bst = xgb.train(params, dtrain, num_round)

        # Make predictions
        test_probabilities = bst.predict(dtest)
        test_predictions = np.argmax(test_probabilities, axis=1)
        del dtrain
        del dtest
        gc.collect()

        # Compute metrics
        accuracy = accuracy_score(test_y, test_predictions)
        balanced_acc = balanced_accuracy_score(test_y, test_predictions)
        roc_auc = roc_auc_score(test_y, test_probabilities, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc_scores = []
        for i in range(test_y_binarized.shape[1]):  # Iterate over each class
            pr_auc = average_precision_score(test_y_binarized[:, i], test_probabilities[:, i])
            pr_auc_scores.append(pr_auc)
        pr_auc = np.mean(pr_auc_scores)
        f2 = fbeta_score(test_y, test_predictions, beta=2, average='weighted')
        mcc = matthews_corrcoef(test_y, test_predictions)

        # Store metrics
        metrics_storage['test_accuracy'].append(accuracy)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(test_y, test_predictions, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()

In [ ]:
run_model("/kaggle/input/ma-bsmote/shuttle_bsmote.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/50 started...

Aggregated Metrics for 1 runs:
test_accuracy: 0.9994
balanced_accuracy: 0.9998
roc_auc: 1.0000
pr_auc: 0.9958
f2: 0.9994
mcc: 0.9983
0_precision: 1.0000
0_recall: 0.9995
0_f1-score: 0.9997
0_support: 9117.0000
1_precision: 0.9091
1_recall: 1.0000
1_f1-score: 0.9524
1_support: 10.0000
2_precision: 0.8718
2_recall: 1.0000
2_f1-score: 0.9315
2_support: 34.0000
3_precision: 1.0000
3_recall: 0.9989
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 0.9985
4_recall: 1.0000
4_f1-score: 0.9992
4_support: 653.0000
5_precision: 1.0000
5_recall: 1.0000
5_f1-score: 1.0000
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9994
macro avg_precision: 0.9685
macro avg_recall: 0.9998
macro avg_f1-score: 0.9832
macro avg_support: 11600.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9994
weighted avg_f1-score: 0.9994
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs:
test_accuracy: 0.

In [ ]:
run_model("/kaggle/input/ma-bsmote/covtype_bsmote.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_accuracy: 0.8930
balanced_accuracy: 0.9035
roc_auc: 0.9903
pr_auc: 0.9463
f2: 0.8930
mcc: 0.8288
0_precision: 0.8981
0_recall: 0.8596
0_f1-score: 0.8784
0_support: 42368.0000
1_precision: 0.8949
1_recall: 0.9133
1_f1-score: 0.9040
1_support: 56661.0000
2_precision: 0.9389
2_recall: 0.8897
2_f1-score: 0.9136
2_support: 7151.0000
3_precision: 0.7526
3_recall: 0.9308
3_f1-score: 0.8322
3_support: 549.0000
4_precision: 0.7254
4_recall: 0.8626
4_f1-score: 0.7881
4_support: 1899.0000
5_precision: 0.8193
5_recall: 0.9047
5_f1-score: 0.8599
5_support: 3473.0000
6_precision: 0.9245
6_recall: 0.9639
6_f1-score: 0.9438
6_support: 4102.0000
accuracy: 0.8930
macro avg_precision: 0.8505
macro avg_recall: 0.9035
macro avg_f1-score: 0.8743
macro avg_support: 116203.0000
weighted avg_precision: 0.8941
weighted avg_recall: 0.8930
weighted avg_f1-score: 0.8931
weighted avg_support: 116203.0000
Run 2/50 started...



Aggregated Metrics for 2 runs:
test_accuracy: 0.8922
balanced_accuracy: 0.9030
roc_auc: 0.9901
pr_auc: 0.9450
f2: 0.8921
mcc: 0.8275
0_precision: 0.8981
0_recall: 0.8580
0_f1-score: 0.8776
0_support: 42368.0000
1_precision: 0.8937
1_recall: 0.9132
1_f1-score: 0.9033
1_support: 56661.0000
2_precision: 0.9393
2_recall: 0.8876
2_f1-score: 0.9128
2_support: 7151.0000
3_precision: 0.7593
3_recall: 0.9362
3_f1-score: 0.8385
3_support: 549.0000
4_precision: 0.7204
4_recall: 0.8573
4_f1-score: 0.7829
4_support: 1899.0000
5_precision: 0.8161
5_recall: 0.9050
5_f1-score: 0.8582
5_support: 3473.0000
6_precision: 0.9230
6_recall: 0.9637
6_f1-score: 0.9429
6_support: 4102.0000
accuracy: 0.8922
macro avg_precision: 0.8500
macro avg_recall: 0.9030
macro avg_f1-score: 0.8737
macro avg_support: 116203.0000
weighted avg_precision: 0.8934
weighted avg_recall: 0.8922
weighted avg_f1-score: 0.8923
weighted avg_support: 116203.0000
Run 3/50 started...



Aggregated Metrics for 3 runs:
test_accuracy: 0.8925
balanced_accuracy: 0.9042
roc_auc: 0.9901
pr_auc: 0.9457
f2: 0.8924
mcc: 0.8280
0_precision: 0.8993
0_recall: 0.8568
0_f1-score: 0.8775
0_support: 42368.0000
1_precision: 0.8930
1_recall: 0.9144
1_f1-score: 0.9036
1_support: 56661.0000
2_precision: 0.9409
2_recall: 0.8880
2_f1-score: 0.9137
2_support: 7151.0000
3_precision: 0.7629
3_recall: 0.9375
3_f1-score: 0.8412
3_support: 549.0000
4_precision: 0.7250
4_recall: 0.8620
4_f1-score: 0.7876
4_support: 1899.0000
5_precision: 0.8169
5_recall: 0.9064
5_f1-score: 0.8593
5_support: 3473.0000
6_precision: 0.9244
6_recall: 0.9641
6_f1-score: 0.9438
6_support: 4102.0000
accuracy: 0.8925
macro avg_precision: 0.8517
macro avg_recall: 0.9042
macro avg_f1-score: 0.8752
macro avg_support: 116203.0000
weighted avg_precision: 0.8937
weighted avg_recall: 0.8925
weighted avg_f1-score: 0.8926
weighted avg_support: 116203.0000
Run 4/50 started...



Aggregated Metrics for 4 runs:
test_accuracy: 0.8926
balanced_accuracy: 0.9043
roc_auc: 0.9901
pr_auc: 0.9457
f2: 0.8925
mcc: 0.8281
0_precision: 0.8994
0_recall: 0.8567
0_f1-score: 0.8775
0_support: 42368.0000
1_precision: 0.8927
1_recall: 0.9145
1_f1-score: 0.9035
1_support: 56661.0000
2_precision: 0.9421
2_recall: 0.8887
2_f1-score: 0.9146
2_support: 7151.0000
3_precision: 0.7621
3_recall: 0.9381
3_f1-score: 0.8410
3_support: 549.0000
4_precision: 0.7231
4_recall: 0.8599
4_f1-score: 0.7856
4_support: 1899.0000
5_precision: 0.8191
5_recall: 0.9076
5_f1-score: 0.8611
5_support: 3473.0000
6_precision: 0.9250
6_recall: 0.9647
6_f1-score: 0.9445
6_support: 4102.0000
accuracy: 0.8926
macro avg_precision: 0.8519
macro avg_recall: 0.9043
macro avg_f1-score: 0.8754
macro avg_support: 116203.0000
weighted avg_precision: 0.8938
weighted avg_recall: 0.8926
weighted avg_f1-score: 0.8927
weighted avg_support: 116203.0000
Run 5/50 started...


...


Aggregated Metrics for 20 runs:
test_accuracy: 0.8927
balanced_accuracy: 0.9042
roc_auc: 0.9902
pr_auc: 0.9469
f2: 0.8926
mcc: 0.8282
0_precision: 0.8993
0_recall: 0.8572
0_f1-score: 0.8778
0_support: 42368.0000
1_precision: 0.8930
1_recall: 0.9143
1_f1-score: 0.9035
1_support: 56661.0000
2_precision: 0.9426
2_recall: 0.8886
2_f1-score: 0.9148
2_support: 7151.0000
3_precision: 0.7633
3_recall: 0.9352
3_f1-score: 0.8405
3_support: 549.0000
4_precision: 0.7226
4_recall: 0.8651
4_f1-score: 0.7874
4_support: 1899.0000
5_precision: 0.8180
5_recall: 0.9049
5_f1-score: 0.8593
5_support: 3473.0000
6_precision: 0.9255
6_recall: 0.9640
6_f1-score: 0.9444
6_support: 4102.0000
accuracy: 0.8927
macro avg_precision: 0.8520
macro avg_recall: 0.9042
macro avg_f1-score: 0.8754
macro avg_support: 116203.0000
weighted avg_precision: 0.8939
weighted avg_recall: 0.8927
weighted avg_f1-score: 0.8928
weighted avg_support: 116203.0000
Run 21/50 started...



Aggregated Metrics for 21 runs:
test_accuracy: 0.8925
balanced_accuracy: 0.9040
roc_auc: 0.9902
pr_auc: 0.9468
f2: 0.8924
mcc: 0.8279
0_precision: 0.8990
0_recall: 0.8570
0_f1-score: 0.8775
0_support: 42368.0000
1_precision: 0.8928
1_recall: 0.9141
1_f1-score: 0.9033
1_support: 56661.0000
2_precision: 0.9426
2_recall: 0.8883
2_f1-score: 0.9146
2_support: 7151.0000
3_precision: 0.7633
3_recall: 0.9356
3_f1-score: 0.8407
3_support: 549.0000
4_precision: 0.7222
4_recall: 0.8646
4_f1-score: 0.7870
4_support: 1899.0000
5_precision: 0.8176
5_recall: 0.9048
5_f1-score: 0.8590
5_support: 3473.0000
6_precision: 0.9253
6_recall: 0.9640
6_f1-score: 0.9443
6_support: 4102.0000
accuracy: 0.8925
macro avg_precision: 0.8518
macro avg_recall: 0.9040
macro avg_f1-score: 0.8752
macro avg_support: 116203.0000
weighted avg_precision: 0.8936
weighted avg_recall: 0.8925
weighted avg_f1-score: 0.8925
weighted avg_support: 116203.0000
Run 22/50 started...



Aggregated Metrics for 22 runs:
test_accuracy: 0.8926
balanced_accuracy: 0.9042
roc_auc: 0.9902
pr_auc: 0.9469
f2: 0.8925
mcc: 0.8280
0_precision: 0.8991
0_recall: 0.8572
0_f1-score: 0.8776
0_support: 42368.0000
1_precision: 0.8929
1_recall: 0.9142
1_f1-score: 0.9034
1_support: 56661.0000
2_precision: 0.9426
2_recall: 0.8881
2_f1-score: 0.9145
2_support: 7151.0000
3_precision: 0.7636
3_recall: 0.9357
3_f1-score: 0.8409
3_support: 549.0000
4_precision: 0.7223
4_recall: 0.8654
4_f1-score: 0.7873
4_support: 1899.0000
5_precision: 0.8173
5_recall: 0.9049
5_f1-score: 0.8588
5_support: 3473.0000
6_precision: 0.9255
6_recall: 0.9640
6_f1-score: 0.9444
6_support: 4102.0000
accuracy: 0.8926
macro avg_precision: 0.8519
macro avg_recall: 0.9042
macro avg_f1-score: 0.8753
macro avg_support: 116203.0000
weighted avg_precision: 0.8937
weighted avg_recall: 0.8926
weighted avg_f1-score: 0.8926
weighted avg_support: 116203.0000
Run 23/50 started...



Aggregated Metrics for 23 runs:
test_accuracy: 0.8925
balanced_accuracy: 0.9041
roc_auc: 0.9902
pr_auc: 0.9469
f2: 0.8924
mcc: 0.8280
0_precision: 0.8991
0_recall: 0.8572
0_f1-score: 0.8776
0_support: 42368.0000
1_precision: 0.8929
1_recall: 0.9142
1_f1-score: 0.9034
1_support: 56661.0000
2_precision: 0.9425
2_recall: 0.8877
2_f1-score: 0.9143
2_support: 7151.0000
3_precision: 0.7634
3_recall: 0.9359
3_f1-score: 0.8409
3_support: 549.0000
4_precision: 0.7224
4_recall: 0.8651
4_f1-score: 0.7873
4_support: 1899.0000
5_precision: 0.8169
5_recall: 0.9049
5_f1-score: 0.8587
5_support: 3473.0000
6_precision: 0.9255
6_recall: 0.9639
6_f1-score: 0.9443
6_support: 4102.0000
accuracy: 0.8925
macro avg_precision: 0.8518
macro avg_recall: 0.9041
macro avg_f1-score: 0.8752
macro avg_support: 116203.0000
weighted avg_precision: 0.8937
weighted avg_recall: 0.8925
weighted avg_f1-score: 0.8926
weighted avg_support: 116203.0000
Run 24/50 started...



Aggregated Metrics for 24 runs:
test_accuracy: 0.8924
balanced_accuracy: 0.9039
roc_auc: 0.9902
pr_auc: 0.9468
f2: 0.8923
mcc: 0.8278
0_precision: 0.8989
0_recall: 0.8572
0_f1-score: 0.8776
0_support: 42368.0000
1_precision: 0.8929
1_recall: 0.9140
1_f1-score: 0.9033
1_support: 56661.0000
2_precision: 0.9422
2_recall: 0.8875
2_f1-score: 0.9140
2_support: 7151.0000
3_precision: 0.7639
3_recall: 0.9356
3_f1-score: 0.8411
3_support: 549.0000
4_precision: 0.7227
4_recall: 0.8647
4_f1-score: 0.7874
4_support: 1899.0000
5_precision: 0.8164
5_recall: 0.9048
5_f1-score: 0.8584
5_support: 3473.0000
6_precision: 0.9255
6_recall: 0.9638
6_f1-score: 0.9443
6_support: 4102.0000
accuracy: 0.8924
macro avg_precision: 0.8518
macro avg_recall: 0.9039
macro avg_f1-score: 0.8751
macro avg_support: 116203.0000
weighted avg_precision: 0.8936
weighted avg_recall: 0.8924
weighted avg_f1-score: 0.8925
weighted avg_support: 116203.0000
Run 25/50 started...


In [ ]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_bsmote.csv", "kdd_test.csv", is_string_labels = True, label_mapping=labels_map)

Memory usage of dataframe is 1782.17 MB
Memory usage after optimization is: 443.79 MB
Decreased by 75.1%
Memory usage of dataframe is 197.38 MB
Memory usage after optimization is: 49.15 MB
Decreased by 75.1%
Run 1/50 started...

Aggregated Metrics for 1 runs:
test_accuracy: 0.9999
balanced_accuracy: 0.8197
roc_auc: 0.9999
pr_auc: 0.9212
f2: 0.9999
mcc: 0.9987
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 1.0000
0_support: 194557.0000
1_precision: 1.0000
1_recall: 0.9997
1_f1-score: 0.9998
1_support: 3178.0000
2_precision: 0.9984
2_recall: 0.9996
2_f1-score: 0.9990
2_support: 2496.0000
3_precision: 1.0000
3_recall: 0.9986
3_f1-score: 0.9993
3_support: 2083.0000
4_precision: 0.9978
4_recall: 0.9935
4_f1-score: 0.9957
4_support: 463.0000
5_precision: 1.0000
5_recall: 1.0000
5_f1-score: 1.0000
5_support: 441.0000
6_precision: 0.9901
6_recall: 0.9804
6_f1-score: 0.9852
6_support: 204.0000
7_precision: 1.0000
7_recall: 1.0000
7_f1-score: 1.0000
7_support: 196.0000
8_precision: 1.0000
8_re

In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-bsmote/darknet_bsmote.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9855
roc_auc: 1.0000
pr_auc: 0.9938
f2: 0.9995
mcc: 0.9983
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9996
1_recall: 0.9992
1_f1-score: 0.9994
1_support: 2657.0000
2_precision: 0.9945
2_recall: 0.9978
2_f1-score: 0.9962
2_support: 908.0000
3_precision: 1.0000
3_recall: 0.9962
3_f1-score: 0.9981
3_support: 522.0000
4_precision: 0.9898
4_recall: 0.9932
4_f1-score: 0.9915
4_support: 293.0000
5_precision: 0.9963
5_recall: 0.9963
5_f1-score: 0.9963
5_support: 269.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 116.0000
7_precision: 0.9592
7_recall: 0.8868
7_f1-score: 0.9216
7_support: 53.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9933
macro avg_recall: 0.9855
macro avg_f1-score: 0.9892
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.


Aggregated Metrics for 2 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9855
roc_auc: 1.0000
pr_auc: 0.9935
f2: 0.9995
mcc: 0.9982
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9992
1_f1-score: 0.9995
1_support: 2657.0000
2_precision: 0.9945
2_recall: 0.9972
2_f1-score: 0.9959
2_support: 908.0000
3_precision: 0.9990
3_recall: 0.9952
3_f1-score: 0.9971
3_support: 522.0000
4_precision: 0.9915
4_recall: 0.9949
4_f1-score: 0.9932
4_support: 293.0000
5_precision: 0.9944
5_recall: 0.9963
5_f1-score: 0.9954
5_support: 269.0000
6_precision: 0.9957
6_recall: 1.0000
6_f1-score: 0.9979
6_support: 116.0000
7_precision: 0.9592
7_recall: 0.8868
7_f1-score: 0.9216
7_support: 53.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9927
macro avg_recall: 0.9855
macro avg_f1-score: 0.9889
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.


Aggregated Metrics for 3 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9864
roc_auc: 1.0000
pr_auc: 0.9937
f2: 0.9995
mcc: 0.9984
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9994
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9956
2_recall: 0.9971
2_f1-score: 0.9963
2_support: 908.0000
3_precision: 0.9994
3_recall: 0.9962
3_f1-score: 0.9978
3_support: 522.0000
4_precision: 0.9921
4_recall: 0.9943
4_f1-score: 0.9932
4_support: 293.0000
5_precision: 0.9938
5_recall: 0.9975
5_f1-score: 0.9957
5_support: 269.0000
6_precision: 0.9972
6_recall: 1.0000
6_f1-score: 0.9986
6_support: 116.0000
7_precision: 0.9532
7_recall: 0.8931
7_f1-score: 0.9221
7_support: 53.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9923
macro avg_recall: 0.9864
macro avg_f1-score: 0.9892
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.


Aggregated Metrics for 4 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9862
roc_auc: 1.0000
pr_auc: 0.9934
f2: 0.9995
mcc: 0.9983
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9993
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9956
2_recall: 0.9972
2_f1-score: 0.9964
2_support: 908.0000
3_precision: 0.9995
3_recall: 0.9957
3_f1-score: 0.9976
3_support: 522.0000
4_precision: 0.9923
4_recall: 0.9940
4_f1-score: 0.9932
4_support: 293.0000
5_precision: 0.9935
5_recall: 0.9981
5_f1-score: 0.9958
5_support: 269.0000
6_precision: 0.9957
6_recall: 1.0000
6_f1-score: 0.9979
6_support: 116.0000
7_precision: 0.9547
7_recall: 0.8915
7_f1-score: 0.9219
7_support: 53.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9923
macro avg_recall: 0.9862
macro avg_f1-score: 0.9892
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.

...


Aggregated Metrics for 15 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9851
roc_auc: 1.0000
pr_auc: 0.9940
f2: 0.9995
mcc: 0.9982
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9994
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9953
2_recall: 0.9974
2_f1-score: 0.9963
2_support: 908.0000
3_precision: 0.9997
3_recall: 0.9950
3_f1-score: 0.9974
3_support: 522.0000
4_precision: 0.9909
4_recall: 0.9934
4_f1-score: 0.9922
4_support: 293.0000
5_precision: 0.9941
5_recall: 0.9965
5_f1-score: 0.9953
5_support: 269.0000
6_precision: 0.9966
6_recall: 1.0000
6_f1-score: 0.9983
6_support: 116.0000
7_precision: 0.9462
7_recall: 0.8843
7_f1-score: 0.9141
7_support: 53.0000
8_precision: 0.9911
8_recall: 1.0000
8_f1-score: 0.9955
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9904
macro avg_recall: 0.9851
macro avg_f1-score: 0.9876
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0


Aggregated Metrics for 16 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9853
roc_auc: 1.0000
pr_auc: 0.9940
f2: 0.9995
mcc: 0.9982
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9994
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9954
2_recall: 0.9974
2_f1-score: 0.9964
2_support: 908.0000
3_precision: 0.9998
3_recall: 0.9949
3_f1-score: 0.9973
3_support: 522.0000
4_precision: 0.9911
4_recall: 0.9934
4_f1-score: 0.9922
4_support: 293.0000
5_precision: 0.9942
5_recall: 0.9967
5_f1-score: 0.9955
5_support: 269.0000
6_precision: 0.9957
6_recall: 1.0000
6_f1-score: 0.9979
6_support: 116.0000
7_precision: 0.9459
7_recall: 0.8856
7_f1-score: 0.9146
7_support: 53.0000
8_precision: 0.9917
8_recall: 1.0000
8_f1-score: 0.9958
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9904
macro avg_recall: 0.9853
macro avg_f1-score: 0.9877
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0


Aggregated Metrics for 17 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9852
roc_auc: 1.0000
pr_auc: 0.9940
f2: 0.9995
mcc: 0.9982
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9994
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9954
2_recall: 0.9974
2_f1-score: 0.9964
2_support: 908.0000
3_precision: 0.9998
3_recall: 0.9948
3_f1-score: 0.9973
3_support: 522.0000
4_precision: 0.9912
4_recall: 0.9934
4_f1-score: 0.9923
4_support: 293.0000
5_precision: 0.9941
5_recall: 0.9969
5_f1-score: 0.9955
5_support: 269.0000
6_precision: 0.9960
6_recall: 1.0000
6_f1-score: 0.9980
6_support: 116.0000
7_precision: 0.9455
7_recall: 0.8846
7_f1-score: 0.9139
7_support: 53.0000
8_precision: 0.9922
8_recall: 1.0000
8_f1-score: 0.9960
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9904
macro avg_recall: 0.9852
macro avg_f1-score: 0.9877
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0


Aggregated Metrics for 18 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9852
roc_auc: 1.0000
pr_auc: 0.9940
f2: 0.9995
mcc: 0.9982
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9994
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9955
2_recall: 0.9974
2_f1-score: 0.9965
2_support: 908.0000
3_precision: 0.9997
3_recall: 0.9950
3_f1-score: 0.9973
3_support: 522.0000
4_precision: 0.9913
4_recall: 0.9934
4_f1-score: 0.9923
4_support: 293.0000
5_precision: 0.9942
5_recall: 0.9971
5_f1-score: 0.9957
5_support: 269.0000
6_precision: 0.9962
6_recall: 1.0000
6_f1-score: 0.9981
6_support: 116.0000
7_precision: 0.9452
7_recall: 0.8847
7_f1-score: 0.9138
7_support: 53.0000
8_precision: 0.9926
8_recall: 1.0000
8_f1-score: 0.9963
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9905
macro avg_recall: 0.9852
macro avg_f1-score: 0.9877
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0


Aggregated Metrics for 19 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9853
roc_auc: 1.0000
pr_auc: 0.9940
f2: 0.9995
mcc: 0.9983
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9994
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9956
2_recall: 0.9974
2_f1-score: 0.9965
2_support: 908.0000
3_precision: 0.9997
3_recall: 0.9951
3_f1-score: 0.9974
3_support: 522.0000
4_precision: 0.9914
4_recall: 0.9935
4_f1-score: 0.9925
4_support: 293.0000
5_precision: 0.9943
5_recall: 0.9973
5_f1-score: 0.9958
5_support: 269.0000
6_precision: 0.9955
6_recall: 1.0000
6_f1-score: 0.9977
6_support: 116.0000
7_precision: 0.9470
7_recall: 0.8848
7_f1-score: 0.9147
7_support: 53.0000
8_precision: 0.9918
8_recall: 1.0000
8_f1-score: 0.9959
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9906
macro avg_recall: 0.9853
macro avg_f1-score: 0.9878
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0